In [10]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

## 1 指向指针的指针

In [11]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

# this is a local tensor
x = torch.tensor([1,2,3,4])
print(x)

# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
print(x_ptr)

# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

print(pointer_to_x_ptr)

tensor([1, 2, 3, 4])
(Wrapper)>[PointerTensor | me:42050510610 -> bob:82174302252]
(Wrapper)>[PointerTensor | me:53433269588 -> alice:42050510610]


In [12]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
print(bob._objects)
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
print(alice._objects)

x_ptr = pointer_to_x_ptr.get()
print(x_ptr)



{82174302252: tensor([1, 2, 3, 4])}
{55713210485: tensor([1, 2, 3, 4, 5]), 42050510610: (Wrapper)>[PointerTensor | alice:42050510610 -> bob:82174302252]}
(Wrapper)>[PointerTensor | me:42050510610 -> bob:82174302252]


In [13]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

tensor([1, 2, 3, 4])

## 2 指针链操作

In [5]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [6]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {55713210485: tensor([1, 2, 3, 4, 5])}
alice: {}


In [7]:
x = x.move(alice)

In [8]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {55713210485: tensor([1, 2, 3, 4, 5])}


In [9]:
x

(Wrapper)>[PointerTensor | me:65315350067 -> alice:55713210485]